## 2019-01-01

### 1.1 数据说明

In [1]:
import pandas as pd
import numpy as np
one=pd.read_csv(r'D:\Entertainment\Prediction of Metro Passenger Flow\Metro_train\record_2019-01-01.csv')
#one.columns=['刷卡发生时间','地铁线路id','地铁站id','刷卡设备编号id','进出站状态','用户身份id','用户刷卡类型']
print(one.dtypes)
one.head()

time         object
lineID       object
stationID     int64
deviceID      int64
status        int64
userID       object
payType       int64
dtype: object


,time,lineID,stationID,deviceID,status,userID,payType
0,2019/1/1 2:00,B,27,1354,0,D13f76f42c9a677c4add94d9e480fb5c5,3
1,2019/1/1 2:01,B,5,200,1,D9a337d37d9512184b8e3fd477934b293,3
2,2019/1/1 2:01,B,5,247,0,Dc9e179298617f40b782490c1f3e2346c,3
3,2019/1/1 2:02,B,5,235,0,D9a337d37d9512184b8e3fd477934b293,3
4,2019/1/1 2:03,B,23,1198,0,Dd1cde61886c23fdb7ef1fdb76c9b1234,3


In [2]:
# 拆分时间
ONE=one
ONE['time']=pd.to_datetime(ONE['time'])
ONE['year']=ONE['time'].dt.year
ONE['month']=ONE['time'].dt.month
ONE['day']=ONE['time'].dt.day
ONE['hour']=ONE['time'].dt.hour
ONE['minute']=ONE['time'].dt.minute
bins=[-1,0,10,20,30,40,50,60]
ONE['did']=pd.cut(ONE['minute'].values,bins,right=False)

#### 数据说明：

1、除第一行外，每行包含一条用户的刷卡记录

2、用户身份id，在用户刷卡类型为3时无法唯一标识用户身份。即此userid可能为多人使用，但在一次进出站期间可以视为同一用户。对于
其他取值的用户刷卡类型，对应的用户身份id可以唯一标识一个用户。

3、进出站状态：0为出站，1为进站

### 1.2 数据预处理

#### 1.2.1 以10min时间段统计

In [3]:
first=ONE
first=first.sort_values(by='time')
# 进站人数
FIRST=first.loc[first['status']==1]
IN=FIRST.groupby(['year','month','day','hour','did','stationID'])['userID'].count().reset_index()
# 出站人数
SECOND=first.loc[first['status']==0]
OUT=SECOND.groupby(['year','month','day','hour','did','stationID'])['userID'].count().reset_index()
IN_OUT=pd.merge(IN,OUT,on=['year','month','day','hour','did','stationID'])
IN_OUT.rename(columns={'userID_x':"IN",'userID_y':'OUT'},inplace=True)
IN_OUT.to_csv(r"D:\Entertainment\Prediction of Metro Passenger Flow\one\IN_OUT.csv")
IN_OUT.head()

,year,month,day,hour,did,stationID,IN,OUT
0,2019,1,1,2,"[0, 10)",5,2,3
1,2019,1,1,2,"[0, 10)",27,1,1
2,2019,1,1,2,"[0, 10)",41,1,1
3,2019,1,1,2,"[0, 10)",77,1,1
4,2019,1,1,2,"[10, 20)",31,4,1


#### 1.2.2 移动&固定人员

In [4]:
second=ONE
# 流动人员
FLOW=second.loc[second['payType']==3]  
FLOW.to_csv(r"D:\Entertainment\Prediction of Metro Passenger Flow\one\FLOW.csv")
FLOW=FLOW.groupby(['year','month','day','hour','did','stationID','status'])['userID'].count().unstack()
FLOW_IN_OUT=FLOW.reset_index()
FLOW_IN_OUT.to_csv(r"D:\Entertainment\Prediction of Metro Passenger Flow\one\FLOW_IN_OUT.csv")



# 固定人员
FIX=second.loc[second['payType']!=3]  
FIX.to_csv(r"D:\Entertainment\Prediction of Metro Passenger Flow\one\FIX.csv")

FIX=FIX.groupby(['year','month','day','hour','did','stationID','status'])['userID'].count().unstack()
FIX_IN_OUT=FIX.reset_index()
FIX_IN_OUT.to_csv(r"D:\Entertainment\Prediction of Metro Passenger Flow\one\FIX_IN_OUT.csv")